In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import re
import time
from datetime import date, datetime
# 
import string
import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Alignment, PatternFill, Font, Border, Side, NamedStyle
from openpyxl.utils import get_column_letter

In [ ]:
stime = time.time() 

# 【109 公務資料: 調查名冊聯絡資料更新比對】 >>> 00_調查名冊.csv
Dear 韻丞,

檢附109年主力農家調查名冊，請查收，謝謝．
因為還需要增加一欄調查員及確認那些農戶需要被剔除（排除家庭收支調查樣本戶），所以此檔案之後還會再更新．

另外，亦檢附主力及勞動力調查完訪後之農戶基本資料，目的是為了讓調查員拿到最新農戶資料．

請以農戶ID碰檔更新農戶電話手機及地址，

若無農戶ID者，請以SEQ流水號碰檔更新．然後再請告知我們總共有多少更新的筆數，謝謝．

最後一個附檔是主選樣本名冊範本，有問題再請提出討論，感謝．

統計室 林楓敏 敬上

【刪除】之後新增的欄位 

============================================================================================

【from】: ID代表以調查名冊的ID能碰到主力勞動力裡的ID，SEQ代表ID碰不到但SEQ有，空白則全碰不到 

【tel_u,phone_u,addr_u】: 碰到之後，撈出來的連絡資訊( 年度最新，主力優先) 

【row_u】: 碰到之後，在主力勞動力excel中的列數

【tel_new,phone_new,addr_new】: 
tel_u,phone_u,addr_u跟調查名冊中的連絡資訊比對後，真正使用的值
若後面三個更新欄位不為Y代表沿用，Y代表用主力勞動力 

【更新市內,更新手機,更新地址】: Y 代表撈出的連絡資訊不為空，且不等於調查名冊原來的值

============================================================================================

In [ ]:
# 目錄
folder_src = ['farmer_income_survey', 'resources']
folder_input = folder_src + ['input']
folder_output = folder_src + ['output']
year = '109'
#
folder_csv = folder_input + [year, 'csv']
folder_raw = folder_input + [year, 'raw']
folder_raw_mlin = folder_raw + ['20201006_楓敏_調查名冊']
# 路徑
csv_survey_name = '109年主力農家調查名冊-20201007(給韻丞更新連結編號10680).csv'
csv_update_name = '主力及勞動力調查農戶基本資料(給韻丞).csv'
csv_survey_path = os.path.join(*folder_raw_mlin, csv_survey_name)
csv_update_path = os.path.join(*folder_raw_mlin, csv_update_name)
# 另存excel,csv
now = datetime.now().strftime('%Y%m%d_%H%M%S')
save_xlsx = f'{year}_調查名冊(更新聯絡資訊)_{now}.xlsx'
save_csv = f'01_調查名冊.csv'
save_xlsx_path = os.path.join(*folder_raw_mlin, save_xlsx)
save_csv_path = os.path.join(*folder_csv, save_csv)

In [ ]:
# 讀取csv檔的共同參數
init = {
    # 'engine': 'python',
    'encoding': 'utf-8',
    'sep': ',',
    'keep_default_na': False,  # 放棄自動轉NA
}
# ID欄位的統一字串處理
def SE_upper(se):
    try:
        return se.str.strip().str.replace('[次]', '??').str.replace('[ \u3000　]','').str.upper()  # 去空白，轉大寫
    except:
        print(f'{se.name}: SE_upper有問題，dtype={se.dtype}')
        return se

# 0.讀檔

## 0-1: 調查名冊

In [ ]:
usecols = [
    'V855', '編號', '層別', '樣本套號', '戶長姓名', '市內電話', '分機', '手機號碼', '地址',
    '總銷售收入組距', '非對象註記', '總處連結編號', 'ID', '連結編號', '主備標記', '縣市', '區鄉鎮名稱',
    'adcode', '農戶編號', '可耕作地面積', '主要經營型態', '戶內15歲以上人口',
    '調查員', '刪除'
]
dtype = {col: str for col in usecols}
# 造idx作唯一編號
df_survey = pd.read_csv(csv_survey_path, **init, dtype=dtype, usecols=usecols).reset_index().rename(columns={'V855': 'SEQ', 'index': 'idx'})
# 欄位清洗________________________________________
cols_clean = ['SEQ', '戶長姓名', '市內電話', '分機', '手機號碼', '地址', 'ID', '連結編號', '農戶編號']
for col in cols_clean:
    df_survey[col] = SE_upper(df_survey[col])
# 電話補0________________________________________
cols_tel = ['市內電話', '手機號碼']
for col in cols_tel:
    df_survey[col] = df_survey[col].str.replace('-', '')  # 去 -
    df_survey[col] = df_survey[col].str[0].str.match('[^0]')*'0' + df_survey[col]  # 開頭非0就補0
    df_survey[col].fillna('', inplace=True)  # ''在取str[0]時會變nan
# 無ID,SEQ者補idx
df_survey['ID'] = np.where(df_survey['ID'] == '', 'ZID_'+df_survey['idx'].astype(str), df_survey['ID'])
df_survey['SEQ'] = np.where(df_survey['SEQ'] == '', 'ZSEQ_'+df_survey['idx'].astype(str), df_survey['SEQ'])
# >>> 去除: 農戶編號空者 <<<
where = df_survey['農戶編號'] != ""
df_survey = df_survey[where]
# ===========================================================================
print(df_survey.shape)
df_survey[:3]

In [ ]:
# 手動補ID，讓劉永昌之後00_109c_farmer_main 時，戶號=ID
where = df_survey['戶長姓名'] == '劉永昌'
idx = df_survey[where].index
df_survey['ID'][idx] = 'T102811151'

## 0-2: 主力&勞動力

In [ ]:
cols_map = {
    '調查名稱': 's_name',
    '年度': 'year',
    '農戶ID': 'ID_u',
    '受訪人姓名': 'name',
    '電話1': 'tel1',
    '電話2': 'tel2',
    '地址': 'addr',
    'SEQ流水號': 'SEQ_u',
    'index': 'row'
}
cols_tar = [
    'row',
    'ID_u',
    'SEQ_u',
    'year',
    's_name',
    'tel', 'phone', 'addr',
    'name'  # , 'tel1', 'tel2',
]
dtype = {col: str for col in cols_map}
df_update = pd.read_csv(csv_update_path, **init, dtype=dtype).reset_index().rename(columns=cols_map)
# 欄位全部清洗________________________________________
for col in df_update.columns:
    df_update[col] = SE_upper(df_update[col])
# 電話補0________________________________________
cols_tel = ['tel1', 'tel2']
for col in cols_tel:
    df_update[col] = df_update[col].str.replace('-', '')  # 去 -
    df_update[col] = df_update[col].str[0].str.match('[^0]')*'0' + df_update[col]  # 開頭非0就補0
    df_update[col].fillna('', inplace=True)  # ''在取str[0]時會變nan
# tel 1,2 區分市內/手機
df_update['tel'] = np.where(df_update['tel1'].str.match('^0[^9]'), df_update['tel1'], '')
df_update['tel'] = np.where(df_update['tel2'].str.match('^0[^9]'), df_update['tel2'], df_update['tel'])
df_update['phone'] = np.where(df_update['tel1'].str.match('^09'), df_update['tel1'], '')
df_update['phone'] = np.where(df_update['tel2'].str.match('^09'), df_update['tel2'], df_update['phone'])
# 農戶ID是0的轉''
df_update['ID_u'] = df_update['ID_u'].str.replace('^0$', '')
# 調查名稱排序用
df_update['s_name'] = df_update['s_name'].str.replace('勞動力', '0_L').str.replace('主力', '1_Farmer')
# 無ID,SEQ者補index
df_update['row'] = df_update['row'] + 2
df_update['ID_u'] = np.where(df_update['ID_u'] == '', 'IDu_'+df_update['row'].astype(str), df_update['ID_u'])
df_update['SEQ_u'] = np.where(df_update['SEQ_u'] == '', 'SEQu_'+df_update['row'].astype(str), df_update['SEQ_u'])
#
df_update = df_update[cols_tar]
#
# ===========================================================================
print(df_update.shape)
df_update[:3]

# 1.>>> Join

In [ ]:
df_s = df_survey[['idx', 'ID', 'SEQ']]
df_s[:3]

## 1-1: 先碰 ID

In [ ]:
df_s_ID = df_s.merge(df_update, left_on=['ID'], right_on=['ID_u'], how='left')
df_s_ID = df_s_ID.sort_values(by=['idx', 'year', 's_name']).drop_duplicates(subset=['idx'], keep='last').reset_index(drop=True)
#
where = df_s_ID['ID_u'].notna()
df_s_ID['from_ID'] = np.where(where, 'Y', '')
cols_map = {'tel': 'tel_ID', 'phone': 'phone_ID', 'addr': 'addr_ID', 'row': 'row_ID'}
df_s_ID = df_s_ID[['idx', 'ID', 'SEQ', 'from_ID', 'tel', 'phone', 'addr', 'row']].rename(columns=cols_map).fillna('')
#
print(df_s_ID.shape)
df_s_ID

## 1-2: 再碰 SEQ

In [ ]:
df_s_SEQ = df_s.merge(df_update, left_on=['SEQ'], right_on=['SEQ_u'], how='left')
df_s_SEQ = df_s_SEQ.sort_values(by=['idx', 'year', 's_name']).drop_duplicates(subset=['idx'], keep='last').reset_index(drop=True)
#
where = df_s_SEQ['SEQ_u'].notna()
df_s_SEQ['from_SEQ'] = np.where(where, 'Y', '')
cols_map = {'tel': 'tel_SEQ', 'phone': 'phone_SEQ', 'addr': 'addr_SEQ', 'row': 'row_SEQ'}
df_s_SEQ = df_s_SEQ[['idx', 'ID', 'SEQ', 'from_SEQ', 'tel', 'phone', 'addr', 'row']].rename(columns=cols_map).fillna('')
#
print(df_s_SEQ.shape)
df_s_SEQ

## 1-3: df_survey 結合碰完 ID, SEQ

In [ ]:
where1 = (df_s_ID['from_ID'] == 'Y')
where3 = (df_s_ID['from_ID'] == '') & (df_s_SEQ['from_SEQ'] == 'Y')
where4 = (df_s_ID['from_ID'] == '') & (df_s_SEQ['from_SEQ'] == '')
#
df_survey['from'] = np.where(where1, 'ID', '')
df_survey['from'] = np.where(where3, 'SEQ', df_survey['from'])
# 優先選碰ID來的===================================================================
df_survey['tel_u'] = np.where(where1, df_s_ID['tel_ID'], '')
df_survey['tel_u'] = np.where(where3, df_s_SEQ['tel_SEQ'], df_survey['tel_u'])
#
df_survey['phone_u'] = np.where(where1, df_s_ID['phone_ID'], '')
df_survey['phone_u'] = np.where(where3, df_s_SEQ['phone_SEQ'], df_survey['phone_u'])
#
df_survey['addr_u'] = np.where(where1, df_s_ID['addr_ID'], '')
df_survey['addr_u'] = np.where(where3, df_s_SEQ['addr_SEQ'], df_survey['addr_u'])
#
df_survey['row_u'] = np.where(where1, df_s_ID['row_ID'], '')
df_survey['row_u'] = np.where(where3, df_s_SEQ['row_SEQ'], df_survey['row_u'])

## 1-4: 跟調查名冊原始聯絡資訊比對

In [ ]:
where_tel = (df_survey['tel_u'] != '') & (df_survey['tel_u'] != df_survey['市內電話'])
df_survey['tel_new'] = np.where(where_tel, df_survey['tel_u'], df_survey['市內電話'])
#
where_phone = (df_survey['phone_u'] != '') & (df_survey['phone_u'] != df_survey['手機號碼'])
df_survey['phone_new'] = np.where(where_phone, df_survey['phone_u'], df_survey['手機號碼'])
#
where_addr = (df_survey['addr_u'] != '') & (df_survey['addr_u'] != df_survey['地址'])
df_survey['addr_new'] = np.where(where_addr, df_survey['addr_u'], df_survey['地址'])
#
df_survey['更新市內'] = np.where(where_tel, 'Y', '')
df_survey['更新手機'] = np.where(where_phone, 'Y', '')
df_survey['更新地址'] = np.where(where_addr, 'Y', '')
#
# del df_survey['idx']
# df_survey['SEQ'] = df_survey['SEQ'].str.replace('^ZSEQ_.+', '')
# df_survey['ID'] = df_survey['ID'].str.replace('^ZID_.+', '')
#
df_survey[:3]

## 1-5: 調查名冊另存新檔

In [ ]:
# csv保留ZSEQ,ZID
df_survey.to_csv(save_csv_path, index=False)

In [ ]:
del df_survey['idx']
df_survey['SEQ'] = df_survey['SEQ'].str.replace('^ZSEQ_.+', '')
df_survey['ID'] = df_survey['ID'].str.replace('^ZID_.+', '')
# 
df_survey.to_excel(save_xlsx_path, index=False)

In [ ]:
wb = load_workbook(save_xlsx_path)
sheet = wb.active

In [ ]:
fill_G = PatternFill(start_color='D9D9D9', end_color='D9D9D9', fill_type='solid')
fill_Y = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
side = Side(style='medium')
border = Border(left=side)
font_red = Font(color='FF0000')
#
widths = [
    15.75, 4.88, 4.88, 8.88, 11.63, 15, 4.88, 12.38, 66.5, 18, 11, 13.25,
    11.75, 8.88, 8.88, 8.88, 11, 7.88, 13.25, 21.25, 13.25, 17.75, 8.38, 8.38
]
widths_rest = [8.43, 15, 12.38, 66.5, 8.43, 15, 12.38, 66.5, 9.3, 9.3, 9.3]
ratio = 1.13
#
cols_AX = list(string.ascii_uppercase)[:24]  # A-X
cols_rest = range(25, 36)  # Y-AI
# 調整欄寬
for idx, col in enumerate(cols_AX):
    sheet.column_dimensions[col].width = widths[idx]*ratio
    sheet[f'{col}1'].fill = fill_G
for col in cols_rest:
    sheet.column_dimensions[get_column_letter(col)].width = widths_rest[col-25]*ratio
    sheet.cell(row=1, column=col).fill = fill_Y
# 加垂直粗線及無SEQ紅字
for row in range(1, sheet.max_row+1):
    sheet.cell(row=row, column=25).border = border
    sheet.cell(row=row, column=30).border = border
    sheet.cell(row=row, column=33).border = border
    if not sheet.cell(row=row, column=1).value:
        for col in range(1,36):
            sheet.cell(row=row, column=col).font = font_red
# 凍結
c = sheet['D2']
sheet.freeze_panes = c

In [ ]:
wb.save(save_xlsx_path)

# 2.調查名冊檢查

## 2-1: 【編號】流水號不順

查到一筆。excel中，1069列的【編號】為 6668，應為 1068

In [ ]:
# 名冊中的【編號】有無流水號
idx_pair = df_survey['編號'].to_frame().astype(int).reset_index().rename(columns={'index':'idx'})
idx_pair['idx'] = idx_pair['idx'] + 1
idx_pair['D'] = idx_pair['idx'] - idx_pair['編號']
# 
idx_pair[idx_pair.D!=0]

## 2-2: 同ID 同套號

In [ ]:
# 同ID 同套號
cols = ['樣本套號','ID']
df_pair = df_survey[cols]
IDs = df_pair[df_pair.duplicated() & (df_pair.ID!="")].ID
df_survey[df_survey.ID.isin(IDs)][cols+['編號']].sort_values(by=['ID','樣本套號'])

## 2-3: 主力勞動力缺聯絡資訊

In [ ]:
# 主力勞動力缺聯絡資訊
where = True
where &= df_update.tel==''
where |= df_update.phone==''
where |= df_update.addr==''
# 
df_update[where]

In [ ]:
df_update.shape

## 2-4: 主備選確認

In [ ]:
cols = ['ID', '樣本套號', '農戶編號', '連結編號', '主備標記']
df_tmp = df_survey[cols]
df_tmp

In [ ]:
where = df_tmp['農戶編號'] == ''
df_tmp[where]

In [ ]:
where = df_tmp['樣本套號'] != df_tmp['農戶編號'].str[-1]
df_tmp[where]

In [ ]:
where = df_tmp['農戶編號'].str[-5:] != df_tmp['連結編號'].str.zfill(5)
df_tmp[where]

In [ ]:
where = df_tmp['樣本套號'] != df_tmp['連結編號'].str[-1]
df_tmp[where]

In [ ]:
where = df_tmp['樣本套號'].str.match('0').astype(object) * '*' != df_tmp['主備標記']
df_tmp[where]

In [ ]:
where = df_tmp['ID'] == ""
df_tmp[where]

In [ ]:
time.time() - stime